In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers, optimizers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from scipy import io
df = pd.read_excel(r'0528_transient sintering_measurement dataset.xlsx', sheet_name='Sheet2')

In [2]:
df = df.dropna()


In [3]:
X = pd.DataFrame(df, columns = ['총전류', '60V','120V','임팩터'])
Y = pd.DataFrame(df, columns = ['전기이동도 입경','공기역학적 입경','밀도'])
print(X)
print(Y)

          총전류       60V      120V       임팩터
0     128.750  0.605437  0.221553  0.421553
1     128.425  0.606969  0.218221  0.424178
2     128.525  0.597160  0.222525  0.422875
3     126.200  0.619849  0.233162  0.422544
4     127.500  0.612941  0.229020  0.419020
...       ...       ...       ...       ...
1032   26.325  0.304609  0.606555  0.698955
1033   75.075  0.269292  0.572012  0.731269
1034   46.025  0.246797  0.521420  0.754481
1035   76.600  0.204238  0.436828  0.806462
1036   41.375  0.190289  0.470633  0.833435

[1037 rows x 4 columns]
      전기이동도 입경    공기역학적 입경        밀도
0     49.53900  119.452408  2.930149
1     49.53900  120.019480  2.948605
2     49.12420  117.697687  2.900967
3     49.70700  117.541881  2.857277
4     49.70700  119.605159  2.923820
...        ...         ...       ...
1032  44.34600  229.625781  8.128449
1033  49.58300  248.953904  8.235648
1034  53.50200  262.992716  8.236264
1035  70.12925  319.511938  7.544239
1036  75.99200  338.552021  9.540853

[1

In [4]:
X = np.array(X)

Y=np.array(Y)
#Y[:,0] /= 1000
print(X)
print(Y)

[[128.75         0.60543689   0.2215534    0.4215534 ]
 [128.425        0.60696905   0.21822075   0.42417754]
 [128.525        0.59716009   0.2225248    0.42287493]
 ...
 [ 46.025        0.24679661   0.52142042   0.75448126]
 [ 76.6          0.20423766   0.43682831   0.80646214]
 [ 41.375        0.19028871   0.47063335   0.83343465]]
[[ 49.539      119.45240755   2.93014888]
 [ 49.539      120.01947995   2.94860476]
 [ 49.1242     117.69768729   2.9009673 ]
 ...
 [ 53.502      262.9927164    8.23626382]
 [ 70.12925    319.51193834   7.54423931]
 [ 75.992      338.55202081   9.54085338]]


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size =0.1, random_state=42)

In [6]:
from sklearn.preprocessing import RobustScaler, StandardScaler

robust_scaler = RobustScaler()
robust_scaler.fit(X_train)
x_robust_scaled = robust_scaler.transform(X_train)
print(x_robust_scaled)

[[-1.00149402 -0.3544815   1.12663252  0.63047703]
 [-0.09561753  0.33900861 -0.29874142  0.16121262]
 [ 7.18675299 -0.86627043  0.09831111 -0.51772003]
 ...
 [-0.6937251   0.28278516 -0.35754246  0.12864869]
 [-0.55229084  0.52825089 -0.14077438 -0.1281747 ]
 [-0.40338645 -0.92468421  0.09197288  0.00995774]]


In [7]:
from sklearn.decomposition import PCA


#for i in range(n):
#    X_ = x_robust_scaled + Y_train[:,i]
#    pca = PCA(n_components = 3)
#    pca.fit(X_)
#    printcipalcomponents = pca.fit_transform(X_)
#    pca1.append(principalcomponents, axis=1)


pca = PCA(n_components = 3)
pca.fit(x_robust_scaled)
principalComponents = pca.fit_transform(x_robust_scaled)
x_robust_scaled = principalComponents
#x_robust_scaled = np.concatenate((x_robust_scaled, principalComponents), axis=1)
print(x_robust_scaled)


[[-1.84500894 -0.0830151   1.09038598]
 [-0.95990989 -0.6822626  -0.39960801]
 [ 6.37257748 -1.21901811  0.56944856]
 ...
 [-1.53791966 -0.56331168 -0.4301486 ]
 [-1.35159122 -0.29084986 -0.57632283]
 [-1.15154098 -0.16784182  0.60128362]]


In [8]:
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.layers import BatchNormalization

#wandb.init(config=tf.flags.FLAGS, sync_tensorboard=True)

adam = optimizers.Nadam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon= 0.0001, name="Nadam" )

# kernel_initializer=he_normal(), kernel_regularizer = L1L2(l1=0.01, l2=0.01)
    #input_layer = Input(shape=(4,), name='input1')
    #hidden_layer = Dense(30, activation='relu',weights=(w,b))

def mlp_model(input_index, output_index):
    model=Sequential([
    Dense(100, input_shape=(input_index,)), 
    Dense(60, activation='relu'),    
    Dense(80, activation='relu'),    
    Dense(80, activation='relu'),   
    Dense(50, activation='relu'),    
    Dense(60, activation='relu'),    
    Dense(30, activation='relu'),    
    Dense(10, activation = 'relu'),
    Dense(output_index)])
    model.compile(optimizer=adam, loss='mean_squared_error',metrics=[Real_acc])
    return model

def lstm_mlp_model(input_index, output_index):
    model1=Sequential([
         Bidirectional(LSTM(60, input_shape = (1,3), return_sequences=True)),
         Bidirectional(LSTM(80,return_sequences=True)),
         Bidirectional(LSTM(100)),
         Dense(70, activation='relu', kernel_regularizer = L1L2(l1=0.01, l2=0.01)),
         Dense(60, activation='relu', kernel_regularizer = L1L2(l1=0.01, l2=0.01)),         
         Dense(40, activation='relu', kernel_regularizer = L1L2(l1=0.01, l2=0.01)),
         Dense(output_index)])
    model1.compile(optimizer=adam, loss='mean_squared_error',metrics=[Real_acc])
    return model1


def Real_acc(y_true,y_pred):
    real_accuracy=(abs((y_true)-(y_pred))/(y_true))*100
    return real_accuracy

In [9]:
x_robust_scaled = x_robust_scaled.reshape((int(len(x_robust_scaled)),1,3))
m1,_,n1= x_robust_scaled.shape
m2,n2= Y_train.shape
print(m1)

933


In [ ]:
m1,n1= x_robust_scaled.shape
m2,n2= Y_train.shape
print(m1)

In [10]:
from sklearn.model_selection import KFold


model = lstm_mlp_model(n1,n2)

In [11]:
model.fit(x_robust_scaled, Y_train, epochs=1000, validation_split=0.1, shuffle=True, verbose=1,  batch_size=5)

W0626 18:13:53.049902  2284 base_layer.py:2068] Layer bidirectional is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1/1000
168/168 [==============================] - 2s 12ms/step - loss: 13320.2900 - Real_acc: 98.9962 - val_loss: 12321.3506 - val_Real_acc: 94.0480
Epoch 2/1000
168/168 [==============================] - 1s 4ms/step - loss: 7899.6538 - Real_acc: 144.5869 - val_loss: 2906.7925 - val_Real_acc: 161.0961
Epoch 3/1000
168/168 [==============================] - 1s 4ms/step - loss: 2557.8811 - Real_acc: 92.9987 - val_loss: 2604.7102 - val_Real_acc: 56.2267
Epoch 4/1000
168/168 [==============================] - 1s 4ms/step - loss: 2345.6501 - Real_acc: 57.3344 - val_loss: 2451.1448 - val_Real_acc: 52.0112
Epoch 5/1000
168/168 [==============================] - 1s 4ms/step - loss: 2181.4333 - Real_acc: 54.9790 - val_loss: 2300.2227 - val_Real_acc: 54.0149
Epoch 6/1000
168/168 [==============================] - 1s 4ms/step - loss: 2028.4584 - Real_acc: 57.0465 - val_loss: 2121.0984 - val_Real_acc: 57.3933
Epoch 7/1000
168/168 [==============================] - 1s 4ms/step - loss: 1850.11

168/168 [==============================] - 1s 4ms/step - loss: 500.8059 - Real_acc: 50.3960 - val_loss: 383.9667 - val_Real_acc: 43.2728
Epoch 56/1000
168/168 [==============================] - 1s 4ms/step - loss: 435.0935 - Real_acc: 46.9806 - val_loss: 345.8547 - val_Real_acc: 40.7762
Epoch 57/1000
168/168 [==============================] - 1s 4ms/step - loss: 378.5192 - Real_acc: 40.7780 - val_loss: 308.0115 - val_Real_acc: 36.4641
Epoch 58/1000
168/168 [==============================] - 1s 4ms/step - loss: 333.9588 - Real_acc: 34.6533 - val_loss: 280.9286 - val_Real_acc: 29.6349
Epoch 59/1000
168/168 [==============================] - 1s 4ms/step - loss: 303.5322 - Real_acc: 30.4482 - val_loss: 260.8450 - val_Real_acc: 25.1420
Epoch 60/1000
168/168 [==============================] - 1s 4ms/step - loss: 286.0856 - Real_acc: 28.8870 - val_loss: 246.4318 - val_Real_acc: 25.5458
Epoch 61/1000
168/168 [==============================] - 1s 4ms/step - loss: 271.0973 - Real_acc: 28.7105 - 

168/168 [==============================] - 1s 4ms/step - loss: 177.4912 - Real_acc: 23.3440 - val_loss: 162.3959 - val_Real_acc: 20.3549
Epoch 110/1000
168/168 [==============================] - 1s 4ms/step - loss: 173.1348 - Real_acc: 23.0673 - val_loss: 162.6492 - val_Real_acc: 19.4515
Epoch 111/1000
168/168 [==============================] - 1s 4ms/step - loss: 174.5024 - Real_acc: 23.3056 - val_loss: 158.4161 - val_Real_acc: 21.9154
Epoch 112/1000
168/168 [==============================] - 1s 4ms/step - loss: 171.9685 - Real_acc: 22.6730 - val_loss: 157.7689 - val_Real_acc: 19.7191
Epoch 113/1000
168/168 [==============================] - 1s 4ms/step - loss: 170.4974 - Real_acc: 22.3896 - val_loss: 157.1428 - val_Real_acc: 21.0527
Epoch 114/1000
168/168 [==============================] - 1s 4ms/step - loss: 166.1525 - Real_acc: 22.8429 - val_loss: 165.9743 - val_Real_acc: 21.6685
Epoch 115/1000
168/168 [==============================] - 1s 4ms/step - loss: 167.2667 - Real_acc: 22.3

168/168 [==============================] - 1s 4ms/step - loss: 54.4879 - Real_acc: 12.5141 - val_loss: 65.4843 - val_Real_acc: 12.8707
Epoch 164/1000
168/168 [==============================] - 1s 4ms/step - loss: 53.9526 - Real_acc: 12.4090 - val_loss: 62.1025 - val_Real_acc: 12.7266
Epoch 165/1000
168/168 [==============================] - 1s 4ms/step - loss: 53.7499 - Real_acc: 12.1663 - val_loss: 64.6615 - val_Real_acc: 12.6492
Epoch 166/1000
168/168 [==============================] - 1s 4ms/step - loss: 52.6367 - Real_acc: 12.0943 - val_loss: 61.1969 - val_Real_acc: 12.6688
Epoch 167/1000
168/168 [==============================] - 1s 4ms/step - loss: 52.2969 - Real_acc: 11.9274 - val_loss: 63.8717 - val_Real_acc: 12.9981
Epoch 168/1000
168/168 [==============================] - 1s 4ms/step - loss: 51.3786 - Real_acc: 12.1727 - val_loss: 63.9276 - val_Real_acc: 13.0136
Epoch 169/1000
168/168 [==============================] - 1s 4ms/step - loss: 50.2658 - Real_acc: 11.8902 - val_los

...

Epoch 772/1000
168/168 [==============================] - 1s 4ms/step - loss: 8.1675 - Real_acc: 3.8396 - val_loss: 8.3804 - val_Real_acc: 4.3614
Epoch 773/1000
168/168 [==============================] - 1s 4ms/step - loss: 8.0396 - Real_acc: 3.7570 - val_loss: 8.2731 - val_Real_acc: 4.4280
Epoch 774/1000
168/168 [==============================] - 1s 4ms/step - loss: 8.0232 - Real_acc: 3.8409 - val_loss: 8.3210 - val_Real_acc: 4.5536
Epoch 775/1000
168/168 [==============================] - 1s 4ms/step - loss: 8.0351 - Real_acc: 3.7636 - val_loss: 8.3182 - val_Real_acc: 4.4105
Epoch 776/1000
168/168 [==============================] - 1s 4ms/step - loss: 8.0769 - Real_acc: 3.7761 - val_loss: 8.2174 - val_Real_acc: 4.2358
Epoch 777/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.9941 - Real_acc: 3.7403 - val_loss: 8.4462 - val_Real_acc: 4.4704
Epoch 778/1000
168/168 [==============================] - 1s 4ms/step - loss: 8.6894 - Real_acc: 3.8000 - val_loss: 8.6455 -

168/168 [==============================] - 1s 4ms/step - loss: 7.6948 - Real_acc: 3.6573 - val_loss: 7.9763 - val_Real_acc: 4.1897
Epoch 829/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.6877 - Real_acc: 3.6872 - val_loss: 7.9597 - val_Real_acc: 4.2451
Epoch 830/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.6592 - Real_acc: 3.6588 - val_loss: 8.3071 - val_Real_acc: 4.4977
Epoch 831/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.7998 - Real_acc: 3.8061 - val_loss: 7.9510 - val_Real_acc: 4.4947
Epoch 832/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.6108 - Real_acc: 3.5924 - val_loss: 8.1073 - val_Real_acc: 4.7697
Epoch 833/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.8151 - Real_acc: 3.6942 - val_loss: 7.9138 - val_Real_acc: 4.2748
Epoch 834/1000
168/168 [==============================] - 1s 5ms/step - loss: 7.6340 - Real_acc: 3.6940 - val_loss: 8.2932 - val_Real_acc: 

168/168 [==============================] - 1s 4ms/step - loss: 7.4075 - Real_acc: 3.6646 - val_loss: 7.9232 - val_Real_acc: 4.2219
Epoch 885/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.4167 - Real_acc: 3.6068 - val_loss: 8.1452 - val_Real_acc: 4.2921
Epoch 886/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.4382 - Real_acc: 3.6542 - val_loss: 7.7280 - val_Real_acc: 4.1626
Epoch 887/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.3343 - Real_acc: 3.6518 - val_loss: 7.5820 - val_Real_acc: 4.2046
Epoch 888/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.4128 - Real_acc: 3.6455 - val_loss: 7.6606 - val_Real_acc: 4.3485
Epoch 889/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.3203 - Real_acc: 3.6514 - val_loss: 7.5354 - val_Real_acc: 4.0764
Epoch 890/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.3197 - Real_acc: 3.5818 - val_loss: 7.6073 - val_Real_acc: 

168/168 [==============================] - 1s 4ms/step - loss: 7.2446 - Real_acc: 3.5504 - val_loss: 7.3764 - val_Real_acc: 4.4105
Epoch 941/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.0559 - Real_acc: 3.5896 - val_loss: 7.3000 - val_Real_acc: 4.0154
Epoch 942/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.0447 - Real_acc: 3.5530 - val_loss: 7.3145 - val_Real_acc: 4.1007
Epoch 943/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.0111 - Real_acc: 3.5803 - val_loss: 7.5383 - val_Real_acc: 4.1534
Epoch 944/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.0159 - Real_acc: 3.5310 - val_loss: 7.3359 - val_Real_acc: 4.2220
Epoch 945/1000
168/168 [==============================] - 1s 4ms/step - loss: 6.9795 - Real_acc: 3.6069 - val_loss: 7.3251 - val_Real_acc: 4.0881
Epoch 946/1000
168/168 [==============================] - 1s 4ms/step - loss: 7.0421 - Real_acc: 3.5124 - val_loss: 7.2858 - val_Real_acc: 

168/168 [==============================] - 1s 4ms/step - loss: 6.7742 - Real_acc: 3.4203 - val_loss: 7.2278 - val_Real_acc: 4.1226
Epoch 997/1000
168/168 [==============================] - 1s 4ms/step - loss: 6.7493 - Real_acc: 3.5259 - val_loss: 7.0715 - val_Real_acc: 3.9579
Epoch 998/1000
168/168 [==============================] - 1s 4ms/step - loss: 6.7832 - Real_acc: 3.4554 - val_loss: 7.1665 - val_Real_acc: 4.1119
Epoch 999/1000
168/168 [==============================] - 1s 4ms/step - loss: 6.7221 - Real_acc: 3.4879 - val_loss: 7.4995 - val_Real_acc: 4.2427
Epoch 1000/1000
168/168 [==============================] - 1s 4ms/step - loss: 6.8117 - Real_acc: 3.4812 - val_loss: 7.2339 - val_Real_acc: 4.0566


In [ ]:
#model.fit(x_robust_scaled, Y_train, epochs=2000, shuffle=True, verbose=1)
scores = []
i=1
for train_index, val_index in KFold(9).split(x_robust_scaled):
    x_train,x_val=x_robust_scaled[train_index],x_robust_scaled[val_index]
    y_train,y_val=Y_train[train_index],Y_train[val_index]

    model.fit(x_train, y_train, epochs=1000, shuffle=True, verbose=1, validation_data = (x_val, y_val), batch_size=10)
    #test_set = pca.fit_transform(robust_scaler.transform(X_test))
    test_set = pca.fit_transform(robust_scaler.transform(X_test))
    test_set = test_set.reshape((int(len(test_set)),1,3))
    scores.append(list(model.evaluate(test_set,Y_test, batch_size = 10, verbose=0)))
    model.save_weights("sintering_learning_"+str(i)+".h5")
    print(scores[i-1])
    i += 1                  

In [ ]:
scores = np.array(scores)
scores[:]

In [ ]:
k = np.argmin(scores[:,1])
print(k+1)
model.load_weights('sintering_learning_'+str(k+1)+'.h5')

In [ ]:
x_all = pca.fit_transform(robust_scaler.transform(X))
x_all = x_all.reshape(int(len(x_all)),1,3)
data1 = model.predict(x_all)
print(data1)
print(Y)

In [12]:
df2 = pd.read_excel(r'dddd.xlsx', sheet_name='Sheet1')
df2 = df2.dropna()
test_input = pd.DataFrame(df2, columns=['총전류','60V','120V','임팩터'])
test_target=pd.DataFrame(df2,columns=['전기이동도 입경','공기역학적 입경','밀도'])
test_input = np.array(test_input)
test_target=np.array(test_target)
test_input = pca.fit_transform(robust_scaler.transform(test_input))
test_input = test_input.reshape(int(len(test_input)),1,3)


data_test = model.predict(test_input)
print(data_test)

[[ 29.502674    62.885525     1.8196684 ]
 [ 33.60846     32.139393     0.93674123]
 [ 38.304844    36.27403      0.93923676]
 [ 47.23855     43.5709       0.90277755]
 [ 42.99855     40.019703     0.9093679 ]
 [ 43.826508    40.73848      0.9104241 ]
 [ 61.092697    56.957726     0.9063567 ]
 [ 87.5874     105.79027      1.2277006 ]
 [ 74.44969    115.90377      1.4012398 ]
 [121.99402    114.29047      0.83337533]
 [137.37254    129.13033      0.825004  ]
 [176.95222    166.37488      0.77241075]
 [ 65.63665     97.14377      1.4513577 ]
 [ 35.57905    200.32944      8.882173  ]
 [ 73.39276    112.70007      1.6806592 ]
 [ 36.824722   220.0149      10.180797  ]
 [ 42.70346    245.65295     10.668107  ]
 [ 46.392822   262.36633     10.973883  ]
 [ 27.752453   124.52691      4.9061213 ]
 [ 34.9389     121.83908      3.7903733 ]
 [ 46.01081    123.636734     3.1735735 ]
 [ 57.674896   126.814606     2.6273885 ]
 [ 97.37465    151.45331      1.5232271 ]
 [ 85.92523    161.67517      2.30

In [13]:
Pred = pd.DataFrame(data_test, columns=['d_em_pred','d_am_pred', 'Density_pred'])
Real = pd.DataFrame(test_target, columns=['d_em','d_am', 'Density' ])
Result3 = pd.concat([Pred, Real], axis =1)

In [14]:
Result3.to_csv('./Result_test.csv',float_format='%.3f',sep=',', na_rep='NaN')

In [ ]:
test_set = pca.fit_transform(robust_scaler.transform(X_test))
test_set = test_set.reshape((int(len(test_set)),1,3))
score= list(model.evaluate(test_set,Y_test, batch_size = 5, verbose=0))
X_test_robust1 = pca.fit_transform(robust_scaler.transform(X_test))
X_test_robust1 = X_test_robust1.reshape((int(len(X_test_robust1)),1,3))
data2 = model.predict(X_test_robust1)
print(score)
print(data2)
print(Y_test)

In [ ]:
Pred = pd.DataFrame(data1, columns=['d_em_pred','d_am_pred', 'Density_pred'])
Real = pd.DataFrame(Y, columns=['d_em','d_am', 'Density' ])
Result = pd.concat([Pred, Real], axis =1)

In [ ]:
Pred = pd.DataFrame(data2, columns=['d_em_pred','d_am_pred', 'Density_pred'])
Real = pd.DataFrame(Y_test, columns=['d_em','d_am', 'Density' ])
Result1 = pd.concat([Pred, Real], axis =1)

In [ ]:
Result.to_csv('./Result_sintering1_all.csv',float_format='%.3f',sep=',', na_rep='NaN')
Result1.to_csv('./Result_sintering1_test.csv',float_format='%.3f',sep=',', na_rep='NaN')

In [ ]:
model.save_weights("sintering_learning_noNtot.h5")
model.save('Sintering_model_noNtot.h5')

In [ ]:
Y1 = pd.DataFrame(df, columns=['수농도'])
Data = np.append(X,data1, axis=1)
Data = np.delete(Data,(1,2,3),axis=1)
Y1 = np.array(Y1)
Y_N = Y1/1000
print(Data)
print(Y_N)

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(Data,Y_N, test_size =0.1, random_state=42)

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler

robust_scaler1 = RobustScaler()
robust_scaler1.fit(X_train1)
x_robust_scaled1 = robust_scaler.transform(X_train1)
print(x_robust_scaled1)

In [ ]:
x_robust_scaled1 = x_robust_scaled1.reshape(len(x_robust_scaled1),1,4)

In [ ]:
m3,n3= x_robust_scaled1.shape
m4,n4= Y_train1.shape
print(m1)

In [ ]:
model1 = mlp_model(n3,n4)

In [ ]:
model1.fit(x_robust_scaled1, Y_train1, epochs=1000, validation_split=0.1, shuffle=True, verbose=1,  batch_size=5)

In [ ]:
x_all1 = robust_scaler1.transform(Data)

Ntot = model1.predict(x_all1)
Ntot = Ntot*1000

print(Ntot)
print(Y1)

In [ ]:
x_test = robust_scaler1.transform(X_test1)
Ntot1 = model1.predict(x_test)
score2= list(model1.evaluate(x_test,Y_test1, batch_size = 5, verbose=0))
Ntot1 = Ntot1*1000
Y_test_N = Y_test1*1000
print(score2)
print(Ntot1)

In [ ]:
data3 = np.append(Ntot,data1,axis=1)
print(data3)
Y_last = np.append(Y1,Y, axis=1)
print(Y_last)

In [ ]:
data4 = np.append(Ntot1,data2, axis=1)
print(data4)
Y_last_test = np.append(Y_test_N,Y_test,axis=1)
print(Y_last_test)

In [ ]:
Pred = pd.DataFrame(data3, columns=['Ntot_pred','d_em_pred','d_am_pred', 'Density_pred'])
Real = pd.DataFrame(Y_last, columns=['Ntot','d_em','d_am', 'Density' ])
Result2 = pd.concat([Pred, Real], axis =1)

In [ ]:
Pred = pd.DataFrame(data4, columns=['Ntot_pred','d_em_pred','d_am_pred', 'Density_pred'])
Real = pd.DataFrame(Y_last_test, columns=['Ntot','d_em','d_am','Density' ])
Result3 = pd.concat([Pred, Real], axis =1)

In [ ]:
Result2.to_csv('./Result_sintering2_all.csv',float_format='%.3f',sep=',', na_rep='NaN')
Result3.to_csv('./Result_sintering2_test.csv',float_format='%.3f',sep=',', na_rep='NaN')

In [ ]:
model.summary()

In [ ]:
model1.summary()

In [ ]:
model1.save_weights("sintering_learning_all"+".h5")
model1.save('Sintering_model_all.h5')